# day four
## part one
simulating a bingo experience

In [1]:
package require Tcl 8.5

8.6.11

In [2]:
package require struct::list 1.8.4
package require struct::matrix 2.0.4
package require generator

0.1

In [3]:
set fl [open 04input]; 
set data [struct::list filter [split [read $fl] \n] {string length}]; close $fl
set boards_tmp [lassign $data numbers]; unset data;

set numbers [split $numbers ,]
puts "numbers: $numbers"

set boards {}
foreach {0 1 2 3 4} [struct::list mapfor r $boards_tmp { struct::list filter [split $r " "] {string length} }] {
    set tmp [struct::matrix]
    $tmp deserialize [list 5 5 [list $0 $1 $2 $3 $4]]
    #puts "[$tmp format 2string]\n"
    lappend boards $tmp
}

proc print_grid {boards} {
    foreach {0 1 2 3 4 5 6 7 8 9} [struct::list mapfor bb $boards { $bb format 2string }] {
        foreach 0 [split $0 \n] 1 [split $1 \n] 2 [split $2 \n] 3 [split $3 \n] 4 [split $4 \n] 5 [split $5 \n] 6 [split $6 \n] 7 [split $7 \n] 8 [split $8 \n] 9 [split $9 \n] {
            puts "$0\t$1\t$2\t$3\t$4\t$5\t$6\t$7\t$8\t$9"
        }
        puts {}
    }
}

puts {}
print_grid $boards

numbers: 87 12 53 23 31 70 37 79 95 16 72 9 98 92 5 74 17 60 96 80 75 11 73 33 3 84 81 2 97 93 59 13 77 52 69 83 51 64 48 82 7 49 20 8 36 66 19 0 99 41 91 78 42 40 62 63 57 39 55 47 29 50 58 34 27 43 30 35 22 28 4 14 26 32 10 88 46 65 90 76 38 6 71 67 44 68 86 25 21 24 56 94 18 89 61 15 1 45 54 85

49 0  9  90 8 	52 53 19 56 80	2  60 64 25 55	49 96 0  87 40	97 98 43 36 48	31 62 76 66 93	98 77 12 72 83	16 47 66 85 17	82 9  2  68 0 	29 58 10 50 19
41 88 56 13 6 	94 33 3  78 32	89 1  21 65 41	13 22 10 84 52	6  83 89 41 85	77 11 97 51 32	22 78 75 88 11	54 87 78 21 60	16 72 83 31 5 	47 4  51 22 69
17 11 45 26 75	10 89 66 48 55	6  12 58 11 68	95 30 50 14 1 	69 79 51 0  67	42 88 68 35 72	67 63 84 60 6 	45 44 35 33 32	60 34 92 62 21	66 5  83 82 25
29 62 27 83 36	99 23 88 8  39	23 49 44 91 14	58 17 98 45 38	62 93 30 82 45	29 79 70 85 47	29 95 73 96 53	88 46 27 49 18	13 48 69 59 81	71 23 64 93 14
31 78 1  55 38	76 75 44 79 14	45 52 99 47 63	63 11 74 62 42	87 32 25 34 29	91 3  96 38 14	45 21 66 5

In [4]:
set data [struct::list filter [split "7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7" \n] {string length}]
set boards_tmp [lassign $data test_numbers]; unset data;

set test_numbers [split $test_numbers ,]

set test_boards {}
foreach {0 1 2 3 4} [struct::list mapfor r $boards_tmp { struct::list filter [split $r " "] {string length} }] {
    set tmp [struct::matrix]
    $tmp deserialize [list 5 5 [list $0 $1 $2 $3 $4]]
    lappend test_boards $tmp
}


puts "$test_numbers"
print_grid $test_boards

7 4 9 5 11 17 23 2 0 14 21 24 10 16 13 6 15 25 12 22 18 20 8 19 3 26 1
22 13 17 11 0 	3  15 0  2  22	14 21 17 24 4 							
8  2  23 4  24	9  18 13 17 5 	10 16 15 9  19							
21 9  14 16 7 	19 8  7  25 23	18 8  23 26 20							
6  10 3  18 5 	20 11 10 24 4 	22 11 13 6  5 							
1  12 20 15 19	14 21 16 12 6 	2  0  12 3  7 							



In [5]:
proc all { lst val } { expr {[lsearch -not $lst $val]==-1} } 

proc score { val board mark } {
        set sum 0
        foreach b [struct::list flatten [$board get rect 0 0 4 4]] m [struct::list flatten [$mark get rect 0 0 4 4]] {
            incr sum [expr {$b*$m}]
        }
        expr { $val* $sum }
    }
    
proc bingo_boards { boards } {
    set res {}
    foreach bb $boards {
        set mark [struct::matrix]
        set unit_r { 1 1 1 1 1 }
        $mark deserialize [list 5 5 [list $unit_r $unit_r $unit_r $unit_r $unit_r]]
        lappend res [list $bb $mark 0]
    }
    return $res
}

proc destroy_boards { boards } {
    foreach bb $boards {
        [lindex $bb 1] destroy
    }
}


generator define get_winning { numbers bingo } {
    set boards [bingo_boards $bingo]
    generator finally destroy_boards $boards
        
    foreach val $numbers {
        set winner {}
        foreach bm $boards {
            lassign $bm bb mm
            set pos [$bb search all $val]
            if { [llength $pos] > 0 } {
                lassign $pos pos
                lassign $pos col row
                $mm set cell $col $row 0
                if { [all [$mm get row $row] 0] || [all [$mm get column $col] 0] } {
                    lappend winner $bm
                }
            }
        }

        if { $winner != {} } {
            set true_win [lsearch -index 2 -all -inline -integer $winner 0]
            generator yield [list $val $true_win $winner]
            foreach tw $true_win {          
                set to_exclude [lsearch -exact $boards $tw]
                lset tw 2 1
                lset boards $to_exclude $tw
            }
        }
    }
}

::get_winning

In [6]:
set win [get_winning $numbers $boards]
generator next $win res
puts "first to win: $res"

first to win: 3 {{::matrix14 ::matrix117 0}} {{::matrix14 ::matrix117 0}}


In [7]:
proc print_bingo { val board } {
    if { $board == {} } { return "nada"  } 
    lassign $board bb mm removed
    list $removed [score $val $bb $mm] $bb
}

set cnt 1
generator foreach wn $win {
    lassign $wn val head_win brs
    puts -nonewline "$cnt: $val. winner:"
    foreach m $head_win {
        puts -nonewline "[print_bingo $val $m]\t"
    }
    puts -nonewline "other:"
    foreach m $brs {
        puts -nonewline "[print_bingo $val $m]\t"
    }
    puts -
    incr cnt
}

1: 59. winner:0 33040 ::matrix46	other:0 33040 ::matrix46	-
2: 13. winner:0 9867 ::matrix21	other:0 9867 ::matrix21	-
3: 69. winner:0 42849 ::matrix54	other:0 42849 ::matrix54	-
4: 83. winner:0 44986 ::matrix7	0 33698 ::matrix9	other:0 44986 ::matrix7	0 33698 ::matrix9	-
5: 51. winner:0 29478 ::matrix30	other:0 29478 ::matrix30	-
6: 64. winner:0 38400 ::matrix15	0 38336 ::matrix76	0 26880 ::matrix92	other:0 38400 ::matrix15	0 38336 ::matrix76	0 26880 ::matrix92	-
7: 48. winner:other:1 17184 ::matrix9	-
8: 82. winner:0 42476 ::matrix73	other:1 44936 ::matrix21	1 35014 ::matrix46	0 42476 ::matrix73	-
9: 7. winner:other:1 1883 ::matrix9	-
10: 49. winner:0 22246 ::matrix22	other:0 22246 ::matrix22	-
11: 8. winner:0 5904 ::matrix27	0 4688 ::matrix51	0 5272 ::matrix89	other:0 5904 ::matrix27	0 4688 ::matrix51	0 5272 ::matrix89	-
12: 36. winner:0 21564 ::matrix12	0 15804 ::matrix60	0 17784 ::matrix86	other:0 21564 ::matrix12	0 15804 ::matrix60	0 17784 ::matrix86	-
13: 66. winner:0 31614 ::mat

In [8]:
puts "\033\[46m 1 \033\[0m10\033\[0m"

 1 10
